# Name: Sarim Aeyzaz
# Roll No.: i21-0328
# Assignment 6

# Instruction

1. Don’t plagiarize, Plagiarism would lead to straight zero
2. Clearly mention your NAME and ROLL NO#.
3. Late submissions are not allowed. Start Early.

In [ ]:
#import libraries
import re
import time
import nltk
import string
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.options import Options
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#from selenium.common.exceptions import NoSuchElementException

# Question 1  (10 marks)

Scrape 3 pages of reviews for a single product listing using product URL.<br /> Note: Product must have more than 1 review page. The code must be generic and should work for every product listing URL, with proper checks in place.

In [ ]:
url = 'https://www.daraz.pk/products/100-i217238343-s1455015550.html?'
url2 = 'https://www.daraz.pk/products/samsung-galaxy-s22-ultra-full-back-protection-vinyl-skinwrap-in-plain-carbon-fiber-and-leather-textures-for-s22ultra-i269071488-s1483526082.html?'
url3 = 'https://www.daraz.pk/products/10-11-6d-i269846065-s1483610918.html?'
url4 = 'https://www.daraz.pk/products/stereo-headphone-earphone-mp3-p47-wireless-bluetooth-42-headset-men-women-i130409640-s1294341998.html?'

df = pd.DataFrame(columns = ['Name', 'Review'])
driver = webdriver.Chrome()
driver.get(url2)

# Close annoying box if any
try:
    driver.find_element(By.XPATH, '/html/body/div[6]/div/div[2]/div/span/i').click()
except:
    pass

for _ in range(3):

    # Go to the reviews and wait for it to load
    driver.execute_script("arguments[0].scrollIntoView(true)", driver.find_element(By.XPATH, '//*[@id="module_product_review"]'))
    time.sleep(5)

    # Loop over each review and add data to dataframe, if there's only image as content, pass 'None' in the Review section
    for review in driver.find_elements(By.CLASS_NAME, "item"):
        driver.execute_script("arguments[0].scrollIntoView(true)", review)
        time.sleep(2)
        try:
            df.loc[len(df)] = [review.find_element(By.CLASS_NAME, "middle").text[3:-18], review.find_element(By.CLASS_NAME, "content").text or 'None']
        except:
            pass

    # Click on the button if possible, else break out (meaning reviews < 15)
    try:
        driver.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]').click()
    except:
        break

driver.quit()
df

,Name,Review
0,3***,very good quality
1,Faizan K,Product received as per ordered. The size of t...
2,Muhammad Y,It's my second purchase with this store. previ...
3,Asad I,Product as described\nWell Packed\nOntime Deli...
4,Hassham D,good product\nrecommend seller
5,AbdurRehman R,Inaccurate Dimensions.\nDon't buy this.\nWaste...
6,DEAD P,None
7,Usman S,"i have ordered 2 accessories from same seller,..."
8,Syed F,The skin doesn't fit at all from the sides and...
9,Mubashar A,Very poor quality protector. It was very grain...


### Question 1 Part b  (10 marks)

Clean the Reviews using NLTK,RE. Remove Punctuations, Smilies and other garbage using any library.

Apply Vader Sentiment to find the sentiment of the review, and append it in a column named "Sentiment".

You can get help of Vader sentiment from this; https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

In [ ]:
sentiment = SentimentIntensityAnalyzer()
emojiPatterns = re.compile(pattern = "["u"\U0001F300-\U0001F5FF"u"\U0001F600-\U0001F64F"u"\U0001F680-\U0001F6FF"u"\U0001F1E0-\U0001F1FF""]+", flags = re.UNICODE)

# Replace \n with " ", remove emojis, remove punctuation
df['Review'] = df['Review'].str.replace('\n', ' ')
df['Review'] = [re.sub(r'[^\w\s]', '', emojiPatterns.sub('', text)) for text in df['Review']]

# Create Sentiment row and declaring positive/negative sentiment based on compound value (as shown in website link)
df['Sentiment'] = ['Positive' if sentiment.polarity_scores(text)['compound'] > 0 else 'Negative' for text in df['Review']]
df

,Name,Review,Sentiment
0,3***,very good quality,Positive
1,Faizan K,Product received as per ordered The size of th...,Negative
2,Muhammad Y,Its my second purchase with this store previou...,Positive
3,Asad I,Product as described Well Packed Ontime Delive...,Positive
4,Hassham D,good product recommend seller,Positive
5,AbdurRehman R,Inaccurate Dimensions Dont buy this Waste of M...,Negative
6,DEAD P,None,Negative
7,Usman S,i have ordered 2 accessories from same seller ...,Negative
8,Syed F,The skin doesnt fit at all from the sides and ...,Negative
9,Mubashar A,Very poor quality protector It was very grainy...,Negative


# Question 2   (20 marks)

Scrape 80 items for a keyword search e.g., 'mobile covers'<br/>Note: Must use "click()" function of webElement, URL changing for pages is not allowed. The code must be generic and should work for every keyword, with proper checks in place. Bonus for also scraping country and Daraz Mall availability.

In [ ]:
query = input('Enter Product: ').strip().replace(' ', '+')
url = f'https://www.daraz.pk/catalog/?q={query}'

df = pd.DataFrame(columns=['Name', 'Price', 'Stars', 'Reviews', 'Country', 'Daraz Mall'])
stars = ['star-icon--k88DV star-0--WgmCt', 'star-icon--k88DV star-1--Do7NZ', 'star-icon--k88DV star-2--fBIsH', 'star-icon--k88DV star-3--zmTQe', 'star-icon--k88DV star-4--hM0en', 'star-icon--k88DV star-5--hXNSC', 'star-icon--k88DV star-6--ezGMb', 'star-icon--k88DV star-7--UNNG4', 'star-icon--k88DV star-8--lQLaV', 'star-icon--k88DV star-9--yMyuX', 'star-icon--k88DV star-10--UQtQk']
driver = webdriver.Chrome()
driver.get(url)

# 80 searches max = 2 pages on daraz, so loop twice
for _ in range(2):

    # Check if there are even any products
    try:
        items = driver.find_elements(By.CLASS_NAME, "info--ifj7U")
    except:
        print("No Products found")

    # Iterate over every listing
    for item in items:

        # Extract Name & Price through class
        name = item.find_element(By.CLASS_NAME, 'title--wFj93').text
        price = int(item.find_element(By.CLASS_NAME, 'price--NVB62').text[4:].replace(',', ''))

        # Extract Reviews or set as 0 if none found
        try:
            reviews = int(item.find_element(By.CLASS_NAME, 'rating__review--ygkUy').text[1:-1])
        except:
            reviews = 0

        # Extract Country (Replace free shipping with China because probability of free shipping is overseas from observation)
        country = item.find_element(By.CLASS_NAME, 'location--eh0Ro').text
        if country == 'Free Shipping':
            country = 'China'

        # Extract Mall availability
        try:
            mall = item.find_element(By.CLASS_NAME, 'ic-dynamic-badge-lazMall')
            mall = 'Yes'
        except:
            mall = 'No'

        # Extract Rating by extracting its box, if none found, set it as 0
        starBox = item.find_element(By.CLASS_NAME, 'rateAndLoc--XWchq')
        try:
            # Go deeper into the divs and find the 5 spans corresponding to the 5 stars rating system
            starBox = starBox.find_element(By.CLASS_NAME, 'rating--ZI3Ol')
            spans = starBox.find_elements(By.TAG_NAME, 'span')[0:-1]

            # Assign each star's class value to an int using the list in the beginning and add the integers up
            # e.g sum([10,10,10,3,0])/10 = 3.3
            rating = sum([stars.index(span.find_element(By.TAG_NAME, 'i').get_attribute('class')) for span in spans])/10
        except:
            rating = 0

        # Add all the data into dataframe
        df.loc[len(df)] = [name, price, rating, reviews, country, mall]

    # If item listings = 40, then go to the next page since this one is full, else back out.
    if len(df) == 40:
        try: # Just incase we have exactly 40 items
            driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div/ul/li[9]/a').click()
        except:
            break
    else:
        break

driver.quit()
df

Enter Product: Mobile covers


,Name,Price,Stars,Reviews,Country,Daraz Mall
0,Hontinga for Samsung A31 Case Luxury 6D Platin...,506,4.7,60,China,Yes
1,Hontinga for Huawei Y9 Prime 2019 Case Luxury ...,441,4.9,147,China,Yes
2,Hontinga for Samsung Galaxy A03 Back Cover Lux...,445,5.0,40,China,Yes
3,Hontinga for OPPO F11 Pro Back Cover Luxury 6D...,524,4.6,17,China,Yes
4,Underwater Waterproof Rainproof Mobile Case PV...,109,4.8,327,Pakistan,No
...,...,...,...,...,...,...
75,Hontinga for Xiaomi Redmi 8 Back Cover Luxury ...,445,4.9,11,China,Yes
76,TA for OPPO F17 Pro A93 2020 Back Cover Luxury...,445,4.9,56,China,No
77,Jizetin for OPPO A52 A92 Back Cover Luxury 6D ...,445,4.9,23,China,Yes
78,Hontinga for Vivo Y33S Y21 2021 Y21S Y21T Y21A...,524,4.8,115,China,Yes


# Question 3  (30 marks)

<!-- Which Products are on Flash sale? Display Product name, Actual Price, Discounted Price, Discount Percentage, Rating, No. of Reviews, Top 3 reviews displayed.

Display output must be like this. Products may vary. -->

If a user wants to make a store to sell something, lets say "Mobile Covers". He/She wants to see folowing things:
1. Top 10 Sellers store names
2. Daraz Mall Store/ Private store
3. Total "Mobile Covers" listings on his store
4. Avg Prices
5. Avg Units selling
6. Positive Seller Ratings
7. Seller Country

In [ ]:
df2 = pd.DataFrame(columns=['Store Name', 'Daraz Mall/Private', 'Avg Price', 'Listings', 'Avg unit selling', 'Seller Ratings', 'Country', 'Followers'])
query = input('Please enter search term: ').strip().replace(' ', '+')
url = f'https://www.daraz.pk/catalog/?q={query}'

# GO TO TURBO MODE YOOOOOOOOOOOOOOOOOO
capability = DesiredCapabilities().CHROME
capability["pageLoadStrategy"] = "eager"

driver = webdriver.Chrome(desired_capabilities=capability, executable_path='chromedriver')
driver.maximize_window()
driver.get(url)

# Check if there are even any products
try:
    items = driver.find_elements(By.CLASS_NAME, "info--ifj7U")
except:
    print("No Products found")

# Grab list of companies from left side and iterate over companies
companies = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label')

for index in range(len(companies)):

    # Select current company
    companies[index].click()
    time.sleep(5)

    # Get name, listings, country
    name = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[1]/div/div[1]/div/div/a').text
    listings = int(driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[1]/div/div[1]/div/div/span[1]').text.split()[0])

    # Calculating average units sold (<=40 item estimation)
    reviews = 0
    for item in driver.find_elements(By.CLASS_NAME, "info--ifj7U"):
        try:
            reviews = reviews + int(item.find_element(By.CLASS_NAME, 'rating__review--ygkUy').text[1:-1])
        except:
            reviews = 0
    reviews = reviews / len(items)

    # Click actions to get average price (.click() is allowed here)
    # average price = (highest price - lowest price) / 2
    originalUrl = driver.current_url
    driver.get(f'{originalUrl}&sort=priceasc')
    time.sleep(5)
    lowestPrice = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[3]/span').text[3:]
    driver.get(f'{originalUrl}&sort=pricedesc')
    time.sleep(5)
    highestPrice = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[3]/span').text[3:]
    averagePrice = int(highestPrice.replace(',', '')) - int(lowestPrice.replace(',', '')) / 2
    driver.get(originalUrl)
    time.sleep(5)

    # Go to first Product of Company
    driver.get(driver.find_element(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[2]/a').get_attribute('href'))
    time.sleep(5)

    # Remove annoying overseas box if it exists
    try:
        driver.find_element(By.XPATH, '/html/body/div[6]/div/div[2]/div/span/i').click()
    except:
        pass

    # Get country info for company
    try:
        country = driver.find_element(By.XPATH, '//*[@id="module_seller_delivery"]/div/div/div[2]/div[2]/div/div/div/div[1]/div').text
        if 'Overseas' in country:
            country = 'Overseas'
        else:
            country = 'Pakistan'
    except:
        country = 'Pakistan'

    # Go to company website (inside first product)
    driver.get(driver.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[3]/a').get_attribute('href'))
    time.sleep(10)

    # Get followers, ratings and daraz mall info from company website
    followers = int(driver.find_element(By.XPATH, '//*[@id="pi-component-container"]/div/div[2]/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[2]/div').text.split()[0])
    ratings = int(driver.find_element(By.XPATH, '//*[@id="pi-component-container"]/div/div[2]/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[3]').text.split("%")[0])

    try:
        mall = driver.find_element(By.XPATH, '//*[@id="pi-component-container"]/div/div[2]/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[2]/img')
        mall = 'Daraz Mall'
    except:
        mall = 'Private'

    # Go back to original URL
    driver.get(url)
    time.sleep(5)

    # Reset companies list (old list becomes invalid due to session ending when changing pages)
    companies = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label')

    # Add data to dataframe
    df2.loc[len(df2)] = [name, mall, averagePrice, listings, reviews, ratings, country, followers]

driver.quit()
df2

Please enter search term: Mobile covers


,Store Name,Daraz Mall/Private,Avg Price,Listings,Avg unit selling,Seller Ratings,Country,Followers
0,Hontinga,Daraz Mall,541.5,190695,79.925,93,Overseas,30504
1,Jizetin,Daraz Mall,537.0,57862,21.775,94,Overseas,4232
2,UCUC,Daraz Mall,795.5,64519,0.525,93,Overseas,7883
3,FELMYST,Daraz Mall,1599.5,55036,1.100,91,Overseas,1791
4,LENUO,Daraz Mall,1104.5,4714,84.475,91,Overseas,14861
5,Lereach,Daraz Mall,1323.5,2976,1.225,89,Overseas,2655
6,Biruiqu,Private,554.5,6,0.000,88,Overseas,5897
7,SGP Hybrid,Daraz Mall,1174.0,270,0.000,86,Overseas,535
8,Nillkin,Daraz Mall,2878.5,259,0.000,92,Overseas,13146
9,Leweisi,Private,374.0,150,0.000,94,Overseas,380


In [ ]:
# Sort dataframe and trim to just Top 10 or less

final_df = df2.sort_values(by = 'Listings', ascending = False)

try:
    final_df = final_df.loc[:10]
except:
    pass

final_df.reset_index(drop = True, inplace = True)
final_df

,Store Name,Daraz Mall/Private,Avg Price,Listings,Avg unit selling,Seller Ratings,Country,Followers
0,Hontinga,Daraz Mall,541.5,190695,79.925,93,Overseas,30504
1,UCUC,Daraz Mall,795.5,64519,0.525,93,Overseas,7883
2,Jizetin,Daraz Mall,537.0,57862,21.775,94,Overseas,4232
3,FELMYST,Daraz Mall,1599.5,55036,1.100,91,Overseas,1791
4,LENUO,Daraz Mall,1104.5,4714,84.475,91,Overseas,14861
5,Lereach,Daraz Mall,1323.5,2976,1.225,89,Overseas,2655
6,SGP Hybrid,Daraz Mall,1174.0,270,0.000,86,Overseas,535
7,Nillkin,Daraz Mall,2878.5,259,0.000,92,Overseas,13146
8,Leweisi,Private,374.0,150,0.000,94,Overseas,380
9,Mizuma,Private,217.0,25,0.000,87,Pakistan,149
